# Mandelbrot Set

The Mandelbrot set (/ˈmændəlbrɒt/) is the set of complex numbers $c$ for which the function $f_{c}(z)=z^{2}+c$ does not diverge to infinity when iterated from $z=0$, i.e., for which the sequence $f_{c}(0)$, $f_{c}(f_{c}(0))$, etc., remains bounded in absolute value. Its definition is credited to Adrien Douady who named it in tribute to the mathematician Benoit Mandelbrot, a pioneer of fractal geometry.

Images of the Mandelbrot set exhibit an elaborate and infinitely complicated boundary that reveals progressively ever-finer recursive detail at increasing magnifications, making the boundary of the Mandelbrot set a fractal curve. The "style" of this repeating detail depends on the region of the set being examined. Mandelbrot set images may be created by sampling the complex numbers and testing, for each sample point $c$, whether the sequence $f_{c}(0),f_{c}(f_{c}(0)),\dotsc$ goes to infinity. Treating the real and imaginary parts of $c$ as image coordinates on the complex plane, pixels may then be coloured according to how soon the sequence $|f_{c}(0)|,|f_{c}(f_{c}(0))|,\dotsc$ crosses an arbitrarily chosen threshold (the threshold has to be at least 2, but is otherwise arbitrary). If $c$ is held constant and the initial value of $z$ is varied instead, one obtains the corresponding Julia set for the point $c$.

The Mandelbrot set has become popular outside mathematics both for its aesthetic appeal and as an example of a complex structure arising from the application of simple rules. It is one of the best-known examples of mathematical visualization, mathematical beauty, and motif.

Source: [Wikipedia](https://en.wikipedia.org/wiki/Mandelbrot_set)

## Formal Definition & Implementation

The Mandelbrot set is the set of values of c in the complex plane for which the orbit of the critical point $z=0$ under iteration of the quadratic map

$
z_{{n+1}}={z_{n}}^{2}+c
$

remains bounded. We provide a vectorized implementation for improved performance. We use a tool called `numba` that takes our functions and optimizes them so that they can be applied to entire arrays of data. We then rewrite our compute function to work with our special vectorized function. We also have some dashboard components prepared.

In [ ]:
from fractal import *
from components import *

### Setup

The Mandelbrot Set is roughly bounded by the 2.5xx2.5 square with vertices (-2.0, -1.25), (0.5, -1.25), (0.5, 1.25), (-2.0, 1.25), so we will use this as our starting point.

In [ ]:
from bokeh import events
from bokeh.io import output_notebook, show         # import tools for showing in a Jupyter notebook
from bokeh.models import ColumnDataSource, Range1d # import structures for data sources and ranges
from bokeh.plotting import figure                  # import bokeh figures
import panel as pn
pn.extension()

steps_slider = pn.widgets.IntSlider(name='Steps', start=1, end=1000, step=1, value=50)
max_iterations_radio = pn.widgets.RadioButtonGroup(name='Max Iterations', options=[100, 500, 1000])
[x_min_text, x_max_text, y_min_text, y_max_text] = create_bounding_box(-2.0, 0.5, -1.25, 1.25)

def visualize_mandelbrot(steps, max_iterations):
    x_min = float(x_min_text.value)
    x_max = float(x_max_text.value)
    y_min = float(y_min_text.value)
    y_max = float(y_max_text.value)
    
    ms = mcompute(x_min, x_max, y_min, y_max, steps, max_iterations)
    
    source = ColumnDataSource(data={
        'image': [ms],
        'x': [x_min],
        'y': [y_min],
        'dw': [x_max-x_min],
        'dh': [y_max-y_min],
    })
    fig = figure(active_scroll='wheel_zoom')
    fig.image(image='image', x='x', y='y', dw='dw', dh='dh', palette='Spectral11', source=source)

    fig.x_range=Range1d(x_min, x_max)
    fig.y_range=Range1d(y_min, y_max)
    
    fig.x_range.reset_start = -2.0
    fig.x_range.reset_end = 0.5
    fig.y_range.reset_start = -1.25
    fig.y_range.reset_end = 1.25
    
    def lod_callback(*args):
        x_min_text.value = str(fig.x_range.start)
        x_max_text.value = str(fig.x_range.end)
        y_min_text.value = str(fig.y_range.start)
        y_max_text.value = str(fig.y_range.end)
        lod_ms = mcompute(fig.x_range.start, fig.x_range.end, fig.y_range.start, fig.y_range.end, steps_slider.value, max_iterations_radio.value)
        source.data = {
            'image': [lod_ms],
            'x': [fig.x_range.start],
            'y': [fig.y_range.start],
            'dw': [fig.x_range.end-fig.x_range.start],
            'dh': [fig.y_range.end-fig.y_range.start],
        }

    fig.on_event(events.LODEnd, lod_callback)
    fig.on_event(events.Reset, lod_callback)

    return fig

interaction = pn.interact(visualize_mandelbrot, steps=steps_slider, max_iterations=max_iterations_radio, throttled=True)

header = '''# Basic Mandelbrot Set Explorer

This is a basic Mandelbrot Set explorer. We can dive deep into the set and control the resolution of our image

'''

layout = pn.Column(
    pn.pane.Markdown(header),
    pn.Row(
        pn.Column(
            x_min_text,
            x_max_text,
            y_min_text,
            y_max_text,
            *interaction[0]
        ),
        interaction[1][0]
    )
)
layout

## Connection to Julia Sets

The Mandelbrot set is very closely related to the Julia. Instead of varying $c$ as we do in Mandelbrot, we can fix $c$ and then vary $z$.

In [ ]:
jsteps_slider = pn.widgets.IntSlider(name='Steps', start=1, end=1000, step=1, value=50)
jmax_iterations_radio = pn.widgets.RadioButtonGroup(name='Max Iterations', options=[25, 50, 75, 100])
jc_input = pn.widgets.TextInput(name='c', value='0.0')
[jx_min_text, jx_max_text, jy_min_text, jy_max_text] = create_bounding_box(-2.5, 2.5, -2.5, 2.5)

def visualize_julia(c, steps, max_iterations):
    x_min = float(jx_min_text.value)
    x_max = float(jx_max_text.value)
    y_min = float(jy_min_text.value)
    y_max = float(jy_max_text.value)
    
    c = complex(c.replace(' ', '').replace('i','j'))
    
    js = jcompute(c, x_min, x_max, y_min, y_max, steps, max_iterations)
    source = ColumnDataSource(data={
        'image': [js],
        'x': [x_min],
        'y': [y_min],
        'dw': [x_max-x_min],
        'dh': [y_max-y_min],
    })
    fig = figure(active_scroll='wheel_zoom')
    fig.image(image='image', x='x', y='y', dw='dw', dh='dh', palette='Spectral11', source=source)

    fig.x_range=Range1d(x_min, x_max)
    fig.y_range=Range1d(y_min, y_max)
    
    fig.x_range.reset_start = -2.5
    fig.x_range.reset_end = 2.5
    fig.y_range.reset_start = -2.5
    fig.y_range.reset_end = 2.5
    
    def lod_callback(*args):
        jx_min_text.value = str(fig.x_range.start)
        jx_max_text.value = str(fig.x_range.end)
        jy_min_text.value = str(fig.y_range.start)
        jy_max_text.value = str(fig.y_range.end)
        lod_js = jcompute(c, fig.x_range.start, fig.x_range.end, fig.y_range.start, fig.y_range.end, jsteps_slider.value, jmax_iterations_radio.value)
        source.data = {
            'image': [lod_js],
            'x': [fig.x_range.start],
            'y': [fig.y_range.start],
            'dw': [fig.x_range.end-fig.x_range.start],
            'dh': [fig.y_range.end-fig.y_range.start],
        }

    fig.on_event(events.LODEnd, lod_callback)
    fig.on_event(events.Reset, lod_callback)

    return fig

jinteraction = pn.interact(visualize_julia, c=jc_input, steps=jsteps_slider, max_iterations=jmax_iterations_radio, throttled=True)

jheader = '''# Basic Julia Set Explorer

This is a basic Julia Set explorer. We can dive deep into the set and control the resolution of our image

'''

jlayout = pn.Column(
    pn.pane.Markdown(jheader),
    pn.Row(
        pn.Column(
            jx_min_text,
            jx_max_text,
            jy_min_text,
            jy_max_text,
            *jinteraction[0]
        ),
        jinteraction[1][0]
    )
)
jlayout